# This file makes an API call to get the data for all starlink statellites

## 1. dependencies

In [3]:
import requests
import json
import configparser
import time
from datetime import datetime
import pandas as pd
import psycopg2
from Raj_Database_credentials import db_user,db_host,db_password,db_port


#https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/44714,44716/orderby/TLE_LINE1 ASC/EPOCH/2025-03-18--2025-03-22/format/tle					


## 2. Links for authentication with Space Track

In [53]:
uriBase                = "https://www.space-track.org"
requestLogin           = "/ajaxauth/login"

## 3. Read Config file

In [54]:
config = configparser.ConfigParser()
config.read("./SLTrack.ini")
configUsr = config.get("configuration","username")
configPwd = config.get("configuration","password")  
configOut = config.get("configuration","output")
siteCred = {'identity': configUsr, 'password': configPwd}

## 4. Make multiple API request strings for the NORAD_IDs in the ephemeris data

In [55]:
ephemeris_norad_ids = pd.read_csv("Unique_norad_ids_ephemeris.csv")["norad_id"].tolist()

norad_id_batch_strings = []
window_start = 0
window_size = 500

while window_start < len(ephemeris_norad_ids):
    window_end = min(window_start + window_size, len(ephemeris_norad_ids))
    
    # Join IDs in the current window with commas
    ephermeris_norad_string = ",".join(str(norad_id) for norad_id in ephemeris_norad_ids[window_start:window_end])
    
    norad_id_batch_strings.append(ephermeris_norad_string)
    
    window_start += window_size

print(norad_id_batch_strings)




['44714,44716,44717,44718,44719,44720,44721,44723,44724,44725,44726,44732,44734,44735,44736,44737,44740,44741,44744,44747,44748,44751,44752,44753,44758,44759,44761,44762,44763,44765,44767,44768,44771,44772,44914,44917,44921,44922,44923,44925,44927,44930,44932,44933,44934,44937,44940,44941,44942,44945,44949,44952,44954,44961,44966,44968,44969,44970,44972,44973,45044,45047,45048,45052,45054,45057,45058,45060,45061,45062,45064,45067,45068,45071,45072,45073,45074,45075,45080,45081,45083,45084,45085,45087,45088,45090,45092,45093,45094,45095,45096,45098,45100,45179,45181,45182,45183,45184,45185,45186,45187,45188,45189,45190,45192,45194,45196,45197,45198,45201,45204,45205,45206,45207,45210,45212,45217,45219,45221,45222,45223,45225,45226,45227,45228,45229,45230,45232,45233,45235,45236,45237,45360,45361,45362,45366,45367,45368,45369,45370,45373,45374,45375,45376,45377,45379,45381,45383,45384,45386,45387,45388,45389,45390,45394,45395,45396,45397,45398,45399,45401,45403,45404,45405,45406,45407,45

## 4. Making the API requests and download the JSON file




In [56]:
class MyError(Exception):
    def __init__(self,args):
        Exception.__init__(self,"my exception was raised with arguments {0}".format(args))
        self.args = args


combined_jsons = []

with requests.Session() as session:
    # Log in
    resp = session.post(uriBase + requestLogin, data=siteCred)
    if resp.status_code != 200:
        raise MyError(resp, "POST fail on login")
    
    print("Login successful!")

    # Query for Starlink satellites

    for Batch_id,norad_id_batch in enumerate(norad_id_batch_strings,start= 1):

        api_link = f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{norad_id_batch}/orderby/TLE_LINE1 ASC/EPOCH/2025-03-18--2025-03-22/format/json"		

        resp = session.get(api_link)

        if resp.status_code != 200:
            print(resp)
            raise MyError(resp, "GET fail on request for Starlink satellites")

        print(f"Batch {Batch_id}: Data retrieval successful!")

        # Parse response JSON
        retData = json.loads(resp.text)
        combined_jsons += retData

    with open('Starlink_Satellites.json', 'w') as json_file: #save the json file
        json.dump(combined_jsons, json_file, indent=4)

    print("JSON data has been saved as 'Starlink_Satellites.json'")


Login successful!
Batch 1: Data retrieval successful!
Batch 2: Data retrieval successful!
Batch 3: Data retrieval successful!
Batch 4: Data retrieval successful!
Batch 5: Data retrieval successful!
Batch 6: Data retrieval successful!
Batch 7: Data retrieval successful!
Batch 8: Data retrieval successful!
Batch 9: Data retrieval successful!
Batch 10: Data retrieval successful!
Batch 11: Data retrieval successful!
Batch 12: Data retrieval successful!
Batch 13: Data retrieval successful!
Batch 14: Data retrieval successful!
JSON data has been saved as 'Starlink_Satellites.json'


## 5. Create the table for TLE data

In [4]:
import psycopg2

# Establish connection
conn = psycopg2.connect(
    host=db_host,
    dbname="Starlink Training Data",
    user=db_user,
    password=db_password,
    port=db_port
)

cur = conn.cursor()

# Create the TLE_raw table
cur.execute("""
CREATE TABLE IF NOT EXISTS TLE_raw (
    id SERIAL PRIMARY KEY,
    creation_date TIMESTAMP,
    object_name VARCHAR(100),
    object_id VARCHAR(50),
    epoch TIMESTAMP,
    mean_motion DOUBLE PRECISION,
    eccentricity DOUBLE PRECISION,
    inclination DOUBLE PRECISION,
    ra_of_asc_node DOUBLE PRECISION,
    arg_of_pericenter DOUBLE PRECISION,
    mean_anomaly DOUBLE PRECISION,
    NORAD_CAT_ID DOUBLE PRECISION,
    bstar DOUBLE PRECISION,
    mean_motion_dot DOUBLE PRECISION,
    mean_motion_ddot DOUBLE PRECISION,
    semimajor_axis DOUBLE PRECISION,
    period DOUBLE PRECISION,
    apoapsis DOUBLE PRECISION,
    periapsis DOUBLE PRECISION,
    tle_line0 TEXT,
    tle_line1 TEXT,
    tle_line2 TEXT,
    tle_full TEXT
);
""")

# Commit and close
conn.commit()
cur.close()
conn.close()

print("Table TLE_raw has been created.")


Table TLE_raw has been created.


## 6. Populate PostgreSQL database

In [8]:
import psycopg2
import json
from datetime import datetime

# Load JSON data from file
with open("Starlink_Satellites.json", "r") as f:
    tle_data = json.load(f)

# Database connection
conn = psycopg2.connect(
    host=db_host,
    dbname="Starlink Training Data",
    user=db_user,
    password=db_password,
    port=db_port
)

cur = conn.cursor()

# Prepare data for insertion
data_to_insert = []

for idx, tle_dic in enumerate(tle_data, start=1):
    try:
        tle_full = tle_dic["TLE_LINE1"] + "\n" + tle_dic["TLE_LINE2"]
        row = (
            datetime.fromisoformat(tle_dic["CREATION_DATE"]),
            tle_dic["OBJECT_NAME"],
            tle_dic["OBJECT_ID"],
            datetime.fromisoformat(tle_dic["EPOCH"]),
            float(tle_dic["MEAN_MOTION"]),
            float(tle_dic["ECCENTRICITY"]),
            float(tle_dic["INCLINATION"]),
            float(tle_dic["RA_OF_ASC_NODE"]),
            float(tle_dic["ARG_OF_PERICENTER"]),
            float(tle_dic["MEAN_ANOMALY"]),
            float(tle_dic["NORAD_CAT_ID"]),
            float(tle_dic["BSTAR"]),
            float(tle_dic["MEAN_MOTION_DOT"]),
            float(tle_dic["MEAN_MOTION_DDOT"]),
            float(tle_dic["SEMIMAJOR_AXIS"]),
            float(tle_dic["PERIOD"]),
            float(tle_dic["APOAPSIS"]),
            float(tle_dic["PERIAPSIS"]),
            tle_dic["TLE_LINE0"],
            tle_dic["TLE_LINE1"],
            tle_dic["TLE_LINE2"],
            tle_full
        )
        data_to_insert.append(row)
        print(f"TLE {idx} appended.")
    except Exception as e:
        print(f"Error parsing TLE {idx}: {e}")

print("All TLEs parsed. Inserting into database...")

# Updated insert query with tle_full
insert_query = """
INSERT INTO TLE_raw (
    creation_date, object_name, object_id, epoch,
    mean_motion, eccentricity, inclination, ra_of_asc_node,
    arg_of_pericenter, mean_anomaly,NORAD_CAT_ID, bstar,
    mean_motion_dot, mean_motion_ddot,
    semimajor_axis, period, apoapsis, periapsis,
    tle_line0, tle_line1, tle_line2, tle_full
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    cur.executemany(insert_query, data_to_insert)
    conn.commit()
    print(f"{cur.rowcount} rows successfully inserted into TLE_raw.")

except Exception as e:
    conn.rollback()
    print(f"Database insertion failed: {e}")

finally:
    cur.close()
    conn.close()



TLE 1 appended.
TLE 2 appended.
TLE 3 appended.
TLE 4 appended.
TLE 5 appended.
TLE 6 appended.
TLE 7 appended.
TLE 8 appended.
TLE 9 appended.
TLE 10 appended.
TLE 11 appended.
TLE 12 appended.
TLE 13 appended.
TLE 14 appended.
TLE 15 appended.
TLE 16 appended.
TLE 17 appended.
TLE 18 appended.
TLE 19 appended.
TLE 20 appended.
TLE 21 appended.
TLE 22 appended.
TLE 23 appended.
TLE 24 appended.
TLE 25 appended.
TLE 26 appended.
TLE 27 appended.
TLE 28 appended.
TLE 29 appended.
TLE 30 appended.
TLE 31 appended.
TLE 32 appended.
TLE 33 appended.
TLE 34 appended.
TLE 35 appended.
TLE 36 appended.
TLE 37 appended.
TLE 38 appended.
TLE 39 appended.
TLE 40 appended.
TLE 41 appended.
TLE 42 appended.
TLE 43 appended.
TLE 44 appended.
TLE 45 appended.
TLE 46 appended.
TLE 47 appended.
TLE 48 appended.
TLE 49 appended.
TLE 50 appended.
TLE 51 appended.
TLE 52 appended.
TLE 53 appended.
TLE 54 appended.
TLE 55 appended.
TLE 56 appended.
TLE 57 appended.
TLE 58 appended.
TLE 59 appended.
TLE 60